# Projeto 1 - Ciência dos Dados

Nome: Ivan de Alcântara Barbosa Barros

Nome: Jamesson Leandro Paiva Santos

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'guarana_jesus.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo guarana_jesus.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
# Carrega dataset de treino
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,rt @giussilvestre: os éguas que tão cancelando...,0
1,@whindersson @lilwhind guaraná jesus... show d...,1
2,bissexuais tomando guarana jesus,0
3,@whindersson @lilwhind guaraná jesus é tipo al...,0
4,rt @maickjhonn: piaui nem tem guaraná jesus is...,0


In [4]:
# Carrega dataset de teste
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,foda que ela é coca-cola e eu guaraná jesus😫,1
1,@niyeongs @tealcookie eu gostei de pernambuco ...,0
2,rt @botbissexual: bissexuais tomando guarana j...,0
3,a bandeira do estado do cara parece mais uma l...,0
4,rt @marilhera: @inesteticas cajuína meets guar...,0


In [5]:
type(train)

pandas.core.frame.DataFrame

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

### Limpeza do dataset

In [6]:
import re

def clean_tweet(texto):
    """
    Essa função define um padrão de pontuações que serão
    removidas do texto de entrada.
    """
    caracteres = '[\-/!.:?;]'
    padrao = re.compile(caracteres)
    texto_limpo = re.sub(padrao, '', texto)
    return texto_limpo

# Limpeza dos dados (pontuação, '\n', 'rt', letras minúsculas)
for indice in range(320):
    train = train.replace(train['Treinamento'][indice], clean_tweet(train['Treinamento'][indice]))
    train['Treinamento'] = train['Treinamento'].str.replace('[\n]', '')
    if train['Treinamento'][indice][:2] == 'rt':
        train = train.replace(train['Treinamento'][indice], train['Treinamento'][indice][3:])
    train = train.replace(train['Treinamento'][indice], train['Treinamento'][indice].lower())
    

In [7]:
# tweets relevantes
tweets_relevantes = train.loc[(train['Classificação'] == 1), :]

# tweets irrelevantes
tweets_irrelevantes = train.loc[(train['Classificação'] == 0), :]

In [8]:
# Transforma tweets relevantes em uma string
texto_tweets_relevantes = ''
for tweet in tweets_relevantes['Treinamento']:
    tweet = str(tweet)
    texto_tweets_relevantes += tweet

# Série com as palavras do conteúdo relevante
palavras_relevantes = pd.Series(texto_tweets_relevantes.split())

# Calcula frequência relativa de cada palavra
palavras_relevantes = palavras_relevantes.value_counts(True)

# Exibe algumas
palavras_relevantes

guaraná        0.066560
jesus          0.060170
de             0.031416
é              0.030351
e              0.027689
                 ...   
caregamento    0.000532
desde          0.000532
qnd            0.000532
viu,           0.000532
meets          0.000532
Length: 801, dtype: float64

In [9]:
# Transforma tweets irrelevantes em uma string
texto_tweets_irrelevantes = ''
for tweet in tweets_irrelevantes['Treinamento']:
    tweet = str(tweet)
    texto_tweets_irrelevantes += tweet

# Série com as palavras do conteúdo irrelevante (normalize=True)
palavras_irrelevantes = pd.Series(texto_tweets_irrelevantes.split())

# Calcula frequência relativa de cada palavra
palavras_irrelevantes = palavras_irrelevantes.value_counts(True)

# Exibe algumas
palavras_irrelevantes

guaraná      0.060512
jesus        0.048587
do           0.039753
a            0.030477
de           0.029152
               ...   
coe          0.000442
1,85idade    0.000442
todas        0.000442
gerais       0.000442
nego         0.000442
Length: 853, dtype: float64

In [10]:
# Conjunto universo
textao = texto_tweets_relevantes + texto_tweets_irrelevantes

# Palavras do universo
palavras_textao = pd.Series(textao.split())

# Obtém frequência de cada palavra no universo
frequencia_palavras_textao = palavras_textao.value_counts(True)
frequencia_palavras_textao

guaraná                       0.063270
jesus                         0.053852
de                            0.030186
do                            0.027047
e                             0.021975
                                ...   
caregamento                   0.000241
httpstcoamnccvsvywsaudades    0.000241
tbdepois                      0.000241
potável                       0.000241
mimo                          0.000241
Length: 1450, dtype: float64

In [11]:
test['Teste'][2]

'rt @botbissexual: bissexuais tomando guarana jesus'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [12]:
# Limpando dados de teste
# Limpeza dos dados (pontuação, '\n', 'rt', letras minúsculas)
for indice in range(161):
    test = test.replace(test['Teste'][indice], clean_tweet(test['Teste'][indice]))
    test['Teste'] = test['Teste'].str.replace('[\n]', '')
    if test['Teste'][indice][:2] == 'rt':
        test = test.replace(test['Teste'][indice], test['Teste'][indice][3:])
    test = test.replace(test['Teste'][indice], test['Teste'][indice].lower())

In [13]:
# Probabilidade relevante
P_R = len(texto_tweets_relevantes.split()) / len(textao.split())

# Probabilidade irrelevante
P_IR = len(texto_tweets_irrelevantes.split()) / len(textao.split())

print(f'Relevante: {P_R} \nIrrelevante: {P_IR}')

Relevante: 0.4535136440473316 
Irrelevante: 0.546727843516059


In [14]:
P_TdadoR = P_R

# Lista P_TdadoR para cada tweet
P_TdadoR_tweets = []

for tweet in test['Teste']:
    for palavra in tweet.split():
        if palavra not in palavras_relevantes:
            pass
        else:
            P_TdadoR *= palavras_relevantes[palavra]
    P_TdadoR_tweets.append(P_TdadoR)
    # Reseta variável para próximo tweet
    P_TdadoR = P_R

P_TdadoR_tweets

[3.605717642830691e-15,
 4.556500153587885e-34,
 3.9459557723456947e-07,
 2.361100689421665e-46,
 5.149872324993338e-10,
 1.1272418187676993e-17,
 4.105361136704862e-35,
 6.683165487187704e-18,
 2.1277212497942473e-05,
 8.272683295520271e-48,
 5.405989111739045e-27,
 9.776816604474934e-30,
 8.598115012468845e-16,
 1.5158391660358622e-41,
 9.890322805675227e-15,
 2.1629463764972015e-08,
 2.42411455553467e-10,
 3.8829708265125486e-10,
 4.02720015814479e-07,
 1.5428167924189626e-41,
 1.3548000137179434e-41,
 8.7610576219575215e-16,
 5.021737958390706e-23,
 1.9274326768306544e-13,
 3.125635233580923e-12,
 1.3678819338018618e-16,
 6.641622786946815e-32,
 6.919530549050952e-16,
 2.129114092313743e-11,
 2.697170214689744e-27,
 2.4313238585762715e-19,
 2.891149015245982e-14,
 1.619542906447329e-18,
 2.9278812855889207e-49,
 2.9760667927948724e-11,
 1.5844111594697514e-24,
 3.5907358428969015e-18,
 5.512732329012368e-05,
 3.2882964769547456e-05,
 2.85228811793644e-23,
 9.671460226337489e-07,
 4

In [15]:
P_TdadoIR = P_IR

# Lista P_TdadoIR para cada tweet
P_TdadoIR_tweets = []

for tweet in test['Teste']:
    for palavra in tweet.split():
        if palavra not in palavras_irrelevantes:
            pass
        else:
            P_TdadoIR *= palavras_irrelevantes[palavra]
    P_TdadoIR_tweets.append(P_TdadoIR)
    # Reseta variável para próximo tweet
    P_TdadoIR = P_IR

P_TdadoIR_tweets

[3.425329177275184e-15,
 5.3215715959611235e-50,
 6.218932681053281e-07,
 2.0738846666927274e-67,
 0.0016074282598474447,
 1.2736607463997382e-16,
 5.0160962387863675e-39,
 2.996420116037429e-16,
 5.927076851829821e-07,
 2.1907375672270325e-37,
 2.590787504881371e-20,
 1.4811147903266995e-24,
 4.263501443483342e-22,
 8.012482174746966e-46,
 9.844437735673798e-21,
 1.440574327851397e-10,
 3.421364028647068e-10,
 1.4488410082250672e-07,
 1.8753396599969486e-07,
 3.4080805499481883e-38,
 1.0211704637612688e-32,
 2.216268196694457e-12,
 6.822101938648751e-20,
 2.6430760296267817e-12,
 1.442568969228422e-08,
 8.56506986414064e-24,
 2.439394395847294e-33,
 9.882459291696724e-18,
 8.094905771074397e-18,
 3.5018584339436716e-31,
 2.3019252816836875e-16,
 1.7453112048968846e-10,
 2.516203082113107e-19,
 5.558555058202613e-48,
 1.0613938109359484e-07,
 2.2753065337657088e-26,
 9.359086051479852e-18,
 1.9879854803766982e-05,
 1.8459865174926486e-05,
 1.5783874645123434e-21,
 2.216268196694457e-12

In [16]:
# Classificador NB

for indice in range(len(P_TdadoR_tweets)):
    if P_TdadoR_tweets[indice] > P_TdadoIR_tweets[indice]:
        test.loc[indice, 'Naive-Bayes'] = 1
    else:
        test.loc[indice, 'Naive-Bayes'] = 0
        
test.head()

,Teste,Classificação,Naive-Bayes
0,foda que ela é cocacola e eu guaraná jesus😫,1,1.0
1,@niyeongs @tealcookie eu gostei de pernambuco ...,0,1.0
2,@botbissexual bissexuais tomando guarana jesus,0,0.0
3,a bandeira do estado do cara parece mais uma l...,0,1.0
4,@marilhera @inesteticas cajuína meets guaraná ...,0,0.0


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**